In [1]:
import feedparser
import pandas as pd
from datetime import datetime

# URL del feed RSS ufficiale Eurostat
rss_url = "https://ec.europa.eu/eurostat/api/dissemination/catalogue/rss/en/statistics-update.rss"
today_str = datetime.utcnow().strftime('%Y-%m-%d')
csv_file = f"eurostat_updates_{today_str}.csv"

# Parsing feed RSS
feed = feedparser.parse(rss_url)
records = []
for entry in feed.entries:
    records.append({
        'published': entry.published,
        'title': entry.title,
        'description': entry.get('description', '').strip(),
        'link': entry.link
    })

# DataFrame e dataset_id
df = pd.DataFrame(records)
df['dataset_id'] = df['title'].str.split(' - ').str[0].str.strip()

# Riordina le colonne
df = df[['published', 'dataset_id', 'title', 'description', 'link']]
df = df.sort_values('published', ascending=False)

df['json'] = df['dataset_id'].apply(
    lambda x: f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{x}?geo=IT"
)

# Salva su CSV
df.to_csv('./csv/'+csv_file, index=False)

print(df[['published','dataset_id', 'description','link']].head(300).to_markdown(index=False, tablefmt="github"))

| published             | dataset_id     | description                                                                                                                                                                  | link                                                                  |
|-----------------------|----------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------|
| 2025-07-19 11:00:00.0 | EI_ISBU_Q      | Construction index by NACE Rev. 2 activity - quarterly data                                                                                                                  | https://ec.europa.eu/eurostat/databrowser/product/page/EI_ISBU_Q      |
| 2025-07-19 11:00:00.0 | TIPSFS33       | Total financial sector liabilities, by instruments, consolidated - % of GDP            

In [2]:
import pandas as pd
import glob
import os

pd.set_option('display.max_colwidth', None)  # Mostra tutto il contenuto delle celle
pd.set_option('display.max_columns', None)   # Mostra tutte le colonne
pd.set_option('display.width', 0)            # Larghezza automatica (usa l’intera finestra)

# Cartella dove si trovano i file CSV
cartella = r"D:\PKM\Github\Eurostat\codice\csv"
# Trova tutti i file CSV nella cartella
lista_file = glob.glob(os.path.join(cartella, "eurostat_updates_*.csv"))

# Leggi e concatena tutti i CSV
df = pd.concat((pd.read_csv(file) for file in lista_file), ignore_index=True)
df = df[['dataset_id', 'description', 'link', 'json']]
df = df.drop_duplicates(subset='dataset_id')

In [4]:
df[df['description'].str.contains("production", case=False, na=False)].sort_values(by='dataset_id')[['dataset_id', 'description']]

,dataset_id,description
3504,AGPRDMET,Agricultural production methods
1097,APRO_CPNH1,Crop production in national humidity
1098,APRO_CPSH1,Crop production in EU standard humidity
302,APRO_MT_PHEADM,Meat production and foreign trade - head - monthly data
633,EI_ISSPR_M,Production in services growth rates by NACE Rev.2 activity - monthly data
634,EI_ISSP_M,Production in services index by NACE Rev.2 activity - monthly data
635,EI_ISSP_Q,Production in services index by NACE Rev.2 activity - quarterly data
3789,EI_IS_M_VTG,Industrial production - monthly data - vintages from 2021 onwards
8918,MED_EG10,Primary energy production
8845,MED_EG20,Secondary energy production (transformation)


In [40]:
import re

def assegna_macro_argomento(row):
    # Normalizza descrizione: lowercase, trattini unicode -> '-', spazi multipli -> singoli
    descrizione = re.sub(r'[–—]', '-', row['description'].lower())
    descrizione = re.sub(r'\s+', ' ', descrizione).strip()
    dataset_id = row['dataset_id']

    # Conti Nazionali e PIL
    if 'gross value added' in descrizione and 'nace rev' in descrizione:
        return 'Conti Nazionali e PIL'
    elif any(x in descrizione for x in ['gva', 'gdp', 'gross domestic product', 'pil', 'b1gq', 'national accounts']):
        return 'Conti Nazionali e PIL'

    # Lavoro e Occupazione
    elif any(x in descrizione for x in ['employment', 'labour', 'occupazione', 'unemployment', 'lfs', 'labour force']):
        return 'Lavoro e Occupazione'

    # Produzione Industriale
    elif any(x in descrizione for x in ['industrial production', 'production index', 'manifatturiero', 'industry']):
        return 'Produzione Industriale'

    # Prezzi e Inflazione
    elif any(x in descrizione for x in ['hicp', 'harmonised index of consumer prices', 'harmonized index', 'consumer price', 'cpi', 'inflation', 'prezzi']):
        return 'Prezzi e Inflazione'

    # Conti con l’Estero e Bilancia dei Pagamenti
    elif any(x in descrizione for x in ['balance of payments', 'bop']):
        return 'Conti con l’Estero e Bilancia dei Pagamenti'

    # Commercio Estero
    elif any(x in descrizione for x in ['international trade', 'import', 'export', 'extra-eu', 'intra-eu', 'commercio']):
        return 'Commercio Estero'

    # Popolazione e Demografia
    elif any(x in descrizione for x in ['population', 'demography', 'fertility', 'birth rate', 'life expectancy']):
        return 'Popolazione e Demografia'

    # Investimenti
    elif any(x in descrizione for x in ['gross fixed capital formation', 'gfcf', 'investment']):
        return 'Investimenti'

    # Imprese
    elif any(x in descrizione for x in ['enterprise', 'business demography', 'imprese', 'enterprise births', 'sme']):
        return 'Imprese e Dinamica Imprenditoriale'

    # Vendite al Dettaglio
    elif any(x in descrizione for x in ['retail trade', 'retail sales', 'vendite']):
        return 'Vendite al Dettaglio'

    # Finanza e Politica Monetaria
    elif any(x in descrizione for x in [
        'financial', 'interest rate', 'euro area', 'mfi', 'bank', 'monetary',
        'securities', 'loans', 'credit', 'debt securities', 'deposit',
        'investment fund', 'bonds', 'balance sheets of mfis',
        'financial institutions', 'lending', 'money market', 'euribor'
    ]):
        return 'Finanza e Politica Monetaria'

    # Agricoltura e Pesca
    elif any(x in descrizione for x in ['agriculture', 'farming', 'crop', 'livestock', 'fish', 'pesca']):
        return 'Agricoltura e Pesca'

    # Energia e Ambiente
    elif any(x in descrizione for x in ['energy', 'emission', 'co2', 'waste', 'climate', 'ambiente']):
        return 'Energia e Ambiente'

    # Istruzione
    elif any(x in descrizione for x in ['education', 'student', 'teacher', 'school']):
        return 'Istruzione'

    # Sanità
    elif any(x in descrizione for x in ['health', 'hospital', 'disease', 'morbidity']):
        return 'Sanità'

    # Trasporti
    elif any(x in descrizione for x in ['transport', 'logistics', 'traffic', 'freight']):
        return 'Trasporti'

    # Turismo
    elif any(x in descrizione for x in ['tourism', 'nights spent', 'hotel', 'overnight']):
        return 'Turismo'

    # Abitazioni e Edilizia
    elif any(x in descrizione for x in ['housing', 'construction', 'building permits', 'real estate']):
        return 'Abitazioni e Edilizia'

    # Digitale e ICT
    elif any(x in descrizione for x in ['ict', 'digital', 'internet', 'broadband']):
        return 'Digitale e ICT'

    # Reddito e Condizioni Sociali
    elif any(x in descrizione for x in ['income', 'social protection', 'poverty', 'living conditions']):
        return 'Reddito e Condizioni Sociali'

    # Migrazioni e Asilo
    elif any(x in descrizione for x in ['asylum', 'migration', 'migrant', 'residence permit', 'refugee']):
        return 'Migrazioni e Asilo'

    # Fallback su dataset_id
    elif dataset_id.startswith(('MIGR_', 'ASYL_', 'ASY', 'DUB')):
        return 'Migrazioni e Asilo'
    elif dataset_id.startswith('HLTH_') or any(x in dataset_id for x in ['DHC', 'DM', 'CO_BEDOC']):
        return 'Sanità'
    elif dataset_id.startswith(('NRG_', 'NRG_CB_', 'NRG_STK_')):
        return 'Energia e Ambiente'
    elif dataset_id.startswith(('APRO_', 'AGPRDMET', 'ANIMALS')):
        return 'Agricoltura e Pesca'
    elif dataset_id.startswith(('NAIO_', 'NASQ_', 'NAIDSQ_', 'NAIDSA_', 'NAMA_')):
        return 'Conti Nazionali e PIL'
    elif dataset_id.startswith(('BOP_', 'EI_BPM6', 'EBOPS2010')):
        return 'Conti con l’Estero e Bilancia dei Pagamenti'
    elif dataset_id.startswith(('PRC_', 'EI_ISPP', 'EI_HPPI', 'HPI')):
        return 'Prezzi e Inflazione'
    elif dataset_id.startswith(('ENV_', 'WASTE', 'AIR_', 'LANDFP')):
        return 'Energia e Ambiente'
    elif dataset_id.startswith(('SDG_16', 'ASYDCF', 'DUB')):
        return 'Giustizia e Sicurezza'
    elif dataset_id.startswith('TOUR_'):
        return 'Turismo'
    elif dataset_id.startswith(('AVIA_', 'MAR_', 'TRN_')):
        return 'Trasporti'
    elif dataset_id.startswith('DEMO_'):
        return 'Popolazione e Demografia'
    elif dataset_id.startswith(('ILC_', 'SPR_')):
        return 'Reddito e Condizioni Sociali'
    elif dataset_id.startswith(('ISOC_', 'LANGUAGE', 'EDUC_')):
        return 'Istruzione'
    elif dataset_id.startswith(('TIPS', 'TPS', 'TEINA', 'TEC', 'TEIMF', 'TET')):
        return 'Indicatori Tematici e Strutturali'
    elif dataset_id.startswith('YTH_'):
        return 'Istruzione'
    elif dataset_id.startswith(('EI_', 'IRT_', 'MFI', 'ECB_', 'BSI_', 'FSC_', 'BSB_', 'BSX_')):
        return 'Finanza e Politica Monetaria'

    # Residuo
    else:
        return 'Altro'


In [41]:
conteggio = df['macro_argomento'].value_counts()
print(conteggio)


macro_argomento
Altro                                 645
Reddito e Condizioni Sociali          172
Trasporti                             154
Popolazione e Demografia              100
Commercio Estero                       80
Conti Nazionali e PIL                  72
Lavoro e Occupazione                   68
Produzione Industriale                 63
Abitazioni e Edilizia                  54
Imprese e Dinamica Imprenditoriale     48
Prezzi e Inflazione                    39
Energia e Ambiente                     37
Agricoltura e Pesca                    37
Istruzione                             37
Investimenti                           33
Turismo                                27
Sanità                                 20
Digitale e ICT                         15
Vendite al Dettaglio                   14
Finanza e Politica Monetaria           10
Name: count, dtype: int64


In [ ]:
# Raggruppa per macro_argomento in orizzontale
gruppo = df.groupby('macro_argomento')['dataset_id'].apply(list)

# Trasforma in DataFrame orizzontale
max_lunghezza = gruppo.apply(len).max()
df_orizzontale = pd.DataFrame(gruppo.tolist(), index=gruppo.index).T

df_orizzontale

In [37]:
for argomento in sorted(gruppo.index):
    dataset_ids = gruppo[argomento]
    print(f"{argomento}: {', '.join(dataset_ids)}")


Abitazioni e Edilizia: TEIIS500, STS_COPR_M, STS_COPR_Q, EI_ISBR_M, EI_ISBU_M, EI_ISBU_Q, TEIIS550, TEIIS510, STS_COPI_M, STS_COBP_Q, STS_COBP_M, STS_COPI_Q, STS_COPR_A, SPR_EXP_FHO, TIPSHO50, STS_COBP_A, STS_COPI_A, PRC_HPI_OOQ, PRC_HPI_OOINW, PRC_HPI_OOCOW, PRC_HPI_OOA, EI_BSBU_M_R2, TEPSR_LM440, TESPM140, SDG_11_11, SDG_01_50, TESSI303, TESSI302, TESSI301, TESSI300, TESSI166, TESSI165, TESSI164, TESSI163, TESSI162, TESSI161, TESSI160, ILC_LI48, ILC_LI45, ILC_MDHO06Q, ILC_MDHO06D, ILC_MDHO06C, ILC_MDHO06B, ILC_MDHO06A, ILC_LVHO07D, ILC_LVHO07C, ILC_LVHO07B, ILC_LVHO07A, ILC_LVHO08A, ILC_LVHO07E, ILC_LVHO08B, HLTH_DHC030, HLTH_DHC050, HLTH_DHC060
Agricoltura e Pesca: SDG_02_40, EF_MP_SOIL, EF_MP_PRAC, ORG_LSTSPEC, ORG_CROPPRO, ORG_CROPAR, TAG00076, FISH_LD_BE, FISH_LD_EL, FISH_LD_PL, FISH_CA_MAIN, FISH_LD_MAIN, TAG00054, AACT_EAA05, AACT_EAA01, AACT_EAA02, AACT_EAA03, AACT_EAA04, AACT_EAA06, AACT_EAA07, FISH_LD_FR, SDG_02_60, TAG00094, TAG00103, TAG00120, TAI04, FISH_LD_RO, APRO_CPNH1

In [38]:
# Raggruppa i dataset_id per macro argomento
gruppo = df.groupby('macro_argomento')['dataset_id'].apply(list)

# Stampa in formato compatto
for argomento, dataset_ids in gruppo.items():
    print(f"{argomento}: {', '.join(dataset_ids)}")


Abitazioni e Edilizia: TEIIS500, STS_COPR_M, STS_COPR_Q, EI_ISBR_M, EI_ISBU_M, EI_ISBU_Q, TEIIS550, TEIIS510, STS_COPI_M, STS_COBP_Q, STS_COBP_M, STS_COPI_Q, STS_COPR_A, SPR_EXP_FHO, TIPSHO50, STS_COBP_A, STS_COPI_A, PRC_HPI_OOQ, PRC_HPI_OOINW, PRC_HPI_OOCOW, PRC_HPI_OOA, EI_BSBU_M_R2, TEPSR_LM440, TESPM140, SDG_11_11, SDG_01_50, TESSI303, TESSI302, TESSI301, TESSI300, TESSI166, TESSI165, TESSI164, TESSI163, TESSI162, TESSI161, TESSI160, ILC_LI48, ILC_LI45, ILC_MDHO06Q, ILC_MDHO06D, ILC_MDHO06C, ILC_MDHO06B, ILC_MDHO06A, ILC_LVHO07D, ILC_LVHO07C, ILC_LVHO07B, ILC_LVHO07A, ILC_LVHO08A, ILC_LVHO07E, ILC_LVHO08B, HLTH_DHC030, HLTH_DHC050, HLTH_DHC060
Agricoltura e Pesca: SDG_02_40, EF_MP_SOIL, EF_MP_PRAC, ORG_LSTSPEC, ORG_CROPPRO, ORG_CROPAR, TAG00076, FISH_LD_BE, FISH_LD_EL, FISH_LD_PL, FISH_CA_MAIN, FISH_LD_MAIN, TAG00054, AACT_EAA05, AACT_EAA01, AACT_EAA02, AACT_EAA03, AACT_EAA04, AACT_EAA06, AACT_EAA07, FISH_LD_FR, SDG_02_60, TAG00094, TAG00103, TAG00120, TAI04, FISH_LD_RO, APRO_CPNH1

In [42]:
# Raggruppa le descrizioni per macro argomento
gruppo_descrizioni = df.groupby('macro_argomento')['description'].apply(list)

# Stampa ordinato per categoria (alfabetico) e descrizioni (alfabetico)
for argomento in sorted(gruppo_descrizioni.index):
    print(f"{argomento}:")
    for descr in sorted(gruppo_descrizioni[argomento], key=str.lower):
        print(f"- {descr}")
    print()



Abitazioni e Edilizia:
- At-risk-of-poverty rate after deducting housing costs by age and sex
- At-risk-of-poverty rate after deducting housing costs by degree of urbanisation
- Building permits - annual data
- Building permits - annual data
- Building permits - monthly data
- Building permits - monthly data
- Building permits - quarterly data
- Construction - monthly data
- Construction cost of new residential buildings
- Construction growth rates by NACE Rev. 2 activity - monthly data
- Construction index by NACE Rev. 2 activity - monthly data
- Construction index by NACE Rev. 2 activity - quarterly data
- Construction producer prices or costs, new residential buildings - annual data
- Construction producer prices or costs, new residential buildings - monthly data
- Construction producer prices or costs, new residential buildings - quarterly data
- Expenditure on housing function by type of benefit and means-testing
- Housing cost overburden rate
- Housing cost overburden rate by age